In [ ]:
from dotenv import load_dotenv
load_dotenv()

In [ ]:
import datetime as datetime
import glob
import os

import pytz
import numpy as np
import pandas as pd

from datetime import date, datetime, timedelta as td
from sqlalchemy import create_engine

In [ ]:
# functions to convert UTC to EST time zone and extract date/time elements
convert_tz = lambda x: x.to_pydatetime().replace(hour=12,tzinfo=pytz.utc).astimezone(pytz.timezone('US/Eastern'))
get_year = lambda x: convert_tz(x).year
get_month = lambda x: '{}-{:02}'.format(convert_tz(x).year, convert_tz(x).month) #inefficient
get_date = lambda x: '{}-{:02}-{:02}'.format(convert_tz(x).year, convert_tz(x).month, convert_tz(x).day) #inefficient

## Connect to `sqlite` DB

In [ ]:
db_path = os.path.abspath(os.getenv('DB_FILEPATH'))
db_path

In [ ]:
engine = create_engine(f'sqlite:///{db_path}', echo=False)
engine

## Read Data from Shortcuts App Export

In [ ]:
list_of_exports = glob.glob(os.getenv('HEALTH_EXPORTS_DIR') + '/*.txt')
latest_file = max(list_of_exports, key=os.path.getmtime)

In [ ]:
data = pd.read_csv(latest_file, delimiter=';')
data['date'] = pd.to_datetime(data['Date']).map(get_date)
data = data.set_index('date')
data.head(23)

## Merge with New Data

### Active Energy

In [ ]:
activeEnergyDB = pd.read_sql_table(
    'Energy',
    con=engine,
    index_col='date',
    coerce_float=True,
    columns=[
        'date',
        'value'
    ],
    parse_dates=[
        'date',
    ]
) if engine.has_table('Energy') else pd.DataFrame(columns=['date', 'value']).set_index('date')
activeEnergyDB

In [ ]:
activeEnergy = data[['ActiveEnergy']]
activeEnergy.columns = ['value']
pd.concat(
    [
        activeEnergyDB[~activeEnergyDB.index.isin(activeEnergy.index)],
        activeEnergy
    ],
    axis=0,
    join='inner'
).to_sql('Energy', con=engine, if_exists='replace')

### Resting Energy

In [ ]:
restingEnergyDB = pd.read_sql_table(
    'RestingEnergy',
    con=engine,
    index_col='date',
    coerce_float=True,
    columns=[
        'date',
        'value'
    ],
    parse_dates=[
        'date',
    ]
) if engine.has_table('RestingEnergy') else pd.DataFrame(columns=['date', 'value']).set_index('date')
restingEnergyDB

In [ ]:
restingEnergy = data[['RestingEnergy']]
restingEnergy.columns = ['value']
pd.concat(
    [
        restingEnergyDB[~restingEnergyDB.index.isin(restingEnergy.index)],
        restingEnergy
    ],
    axis=0,
    join='inner'
).to_sql('RestingEnergy', con=engine, if_exists='replace')

### RHR

In [ ]:
rhrDB = pd.read_sql_table(
    'RHR',
    con=engine,
    index_col='date',
    coerce_float=True,
    columns=[
        'date',
        'value'
    ],
    parse_dates=[
        'date',
    ]
) if engine.has_table('RHR') else pd.DataFrame(columns=['date', 'value']).set_index('date')
rhrDB

In [ ]:
rhr = data[['RHR']]
rhr.columns = ['value']
pd.concat(
    [
        rhrDB[~rhrDB.index.isin(rhr.index)],
        rhr
    ],
    axis=0,
    join='inner'
).to_sql('RHR', con=engine, if_exists='replace')

### Steps

In [ ]:
stepsDB = pd.read_sql_table(
    'Steps',
    con=engine,
    index_col='date',
    coerce_float=True,
    columns=[
        'date',
        'Steps'
    ],
    parse_dates=[
        'date',
    ]
) if engine.has_table('Steps') else pd.DataFrame(columns=['date', 'Steps']).set_index('date')
stepsDB

In [ ]:
steps = data[['Steps']]
steps.columns = ['Steps']
pd.concat(
    [
        stepsDB[~stepsDB.index.isin(steps.index)],
        steps
    ],
    axis=0,
    join='inner'
).to_sql('Steps', con=engine, if_exists='replace')

### Weight

In [ ]:
weightDB = pd.read_sql_table(
    'Weight',
    con=engine,
    index_col='date',
    coerce_float=True,
    columns=[
        'date',
        'Weight'
    ],
    parse_dates=[
        'date',
    ]
) if engine.has_table('Weight') else pd.DataFrame(columns=['date', 'Weight']).set_index('date')
weightDB

In [ ]:
weight = data[['Weight']]
weight.columns = ['Weight']
pd.concat(
    [
        weightDB[~weightDB.index.isin(weight.index)],
        weight
    ],
    axis=0,
    join='inner'
).to_sql('Weight', con=engine, if_exists='replace')